In [1]:
import pandas as pd 

In [2]:
df_financial = pd.read_excel("../data/FinancialData/BDT_FinConstKZ.xlsx")

/Users/improvise/miniconda/envs/Richthofen/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [5]:
df_financial = df_financial[(df_financial['STPT']==1) & (df_financial['IsNewOrSuspend']==1) & (df_financial['ISBSE']==0)]

In [7]:
df_financial['TobinQ_t1'] = df_financial.groupby(['Symbol'])['TobinQ'].shift(-1)
df_financial

/var/folders/kd/t2wnd4h911v09vtv543hpkg80000gn/T/ipykernel_77661/2745525889.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_financial['TobinQ_t1'] = df_financial.groupby(['Symbol'])['TobinQ'].shift(-1)


,Symbol,ShortName,Enddate,STPT,IsNewOrSuspend,ISBSE,TobinQ,TobinQ_t1
5,000002,万科A,2020-12-31,1,1,0,0.992654,0.929695
16,000002,万科A,2021-12-31,1,1,0,0.929695,0.908089
25,000002,万科A,2022-12-31,1,1,0,0.908089,0.849467
33,000002,万科A,2023-12-31,1,1,0,0.849467,NaN
41,000004,国华网安,2020-12-31,1,1,0,1.637271,2.397518
...,...,...,...,...,...,...,...,...
139358,900953,凯马B,2023-12-31,1,1,0,1.11423,NaN
139367,900957,凌云B股,2020-12-31,1,1,0,1.338059,1.452089
139374,900957,凌云B股,2021-12-31,1,1,0,1.452089,1.397454
139383,900957,凌云B股,2022-12-31,1,1,0,1.397454,1.204864


In [14]:
def getDataOrganized(df:pd.DataFrame) -> pd.DataFrame: 
    
    if 'STPT' in df.columns: 
        df = df[df['STPT'] == 1]
    if 'IsNewOrSuspend' in df.columns: 
        df = df[df['IsNewOrSuspend'] == 1]
    if 'ISBSE' in df.columns: 
        df = df[df['ISBSE'] == 0]
        
    return df 

def getShiftData(df: pd.DataFrame, target_col:str, thscode:str) -> pd.DataFrame: 
    
    if (target_col in df.columns) & (thscode in df.columns): 
        df[f'{target_col}_t1'] = df.groupby(thscode)[target_col].shift(-1)
        return df 
    else: 
        raise ValueError(f"Do not have both the {target_col} and {thscode} in the df.columns")

In [9]:
df_RO = pd.read_excel("../data/FinancialData/CSR_Finidx.xlsx")
df_RO

/Users/improvise/miniconda/envs/Richthofen/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Stkcd,Accper,A100000,T40401,T40801
0,证券代码,统计截止日期,总资产,资产收益率A,净资产收益率A
1,没有单位,没有单位,元,没有单位,没有单位
2,000001,2020-12-31,4468514000000,0.006474,0.079444
3,000001,2021-12-31,4921380000000,0.007383,0.091886
4,000001,2022-12-31,5321514000000,0.008553,0.104712
...,...,...,...,...,...
20317,900955,2021-12-31,1772041329.11,-0.133145,-0.382096
20318,900957,2020-12-31,1008561997.16,0.021673,0.043801
20319,900957,2021-12-31,1023536692.02,0.02701,0.052634
20320,900957,2022-12-31,1066446596.43,0.038026,0.071539


In [16]:
df_RO.rename(columns={"T40401":"ROA", "T40801":"ROE"}, inplace = True)
df_RO = getShiftData(df_RO, 'ROA', 'Stkcd')
df_RO = getShiftData(df_RO, 'ROE', 'Stkcd')

In [19]:
df_financial.columns

Index(['Symbol', 'ShortName', 'Enddate', 'STPT', 'IsNewOrSuspend', 'ISBSE',
       'TobinQ', 'TobinQ_t1'],
      dtype='object')

In [20]:
df_main = df_financial[['Symbol', 'Enddate', 'TobinQ_t1']].merge(df_RO[['Stkcd', 'Accper', 'ROA_t1', 'ROE_t1']], 
                             left_on= ['Symbol', 'Enddate'], right_on = ['Stkcd', 'Accper'], 
                             how = 'outer')

In [21]:
df = pd.read_excel("../data/data_stata4_6_0.25.xlsx")
df

,Unnamed: 0,Corp_code,Pub_year,Corp_industry_code,Mono_idst,Poll_idst,img_num_pages_ratio,img_size_pages_ratio,dist_diag_ratio,word_simi_avg,...,Intersection_A,Intersection_A_sqr,TobinQ,img_num,img_size_avg,PDF_pages,TobinQ_t-0,TobinQ_t-1,TobinQ_t-2,TobinQ_t-3
0,0,1,2020,J66,0,0,-1.108526,-0.850690,-1.386487,0.239547,...,0.368174,-0.408130,1.002502,45,58314.219881,83,1.002502,1.001585,0.976896,1.001915
1,1,8,2022,C37,0,0,-0.481239,0.880235,-0.727693,0.308537,...,0.108048,-0.051997,1.165025,46,215465.342671,35,1.165025,1.222200,1.029485,1.197542
2,2,16,2022,C39,0,0,-0.576226,-0.931491,-0.849546,0.299738,...,0.146731,-0.084550,1.003631,91,34048.336896,76,1.003631,1.045699,1.030459,0.972553
3,3,40,2021,D44,1,0,-0.388391,0.025943,0.244892,0.308726,...,-0.029364,0.011405,0.842710,60,145511.511776,42,0.842710,0.809312,0.831874,0.920839
4,4,40,2022,D44,1,0,0.211256,-0.066994,-0.170544,0.386769,...,-0.013935,-0.002944,0.837492,91,133213.458621,42,0.837492,0.842710,0.809312,0.831874
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
907,933,831039,2022,M74,0,0,0.563307,1.640090,-0.549016,0.518732,...,-0.160425,-0.090369,1.159772,104,342008.029665,40,1.159772,1.283682,NaN,NaN
908,934,831445,2021,C21,0,0,0.690660,-0.479043,0.362501,0.604800,...,0.151421,0.104580,3.313189,102,69320.133859,37,3.313189,1.772118,NaN,NaN
909,935,835640,2022,C39,0,0,-0.009666,-0.140413,-0.958670,-1.512639,...,-0.014018,0.000136,2.492775,36,55868.172382,19,2.492775,3.764790,1.345553,NaN
910,936,838262,2022,C17,0,0,0.278958,-0.616081,0.928865,0.558373,...,0.144683,0.040360,1.128583,99,63437.286708,44,1.128583,NaN,NaN,NaN


In [ ]:
df_main.drop(columns={"Symbol", "Enddate"}, inplace= True)
df_main['Pub_year'] = df_main['Accper'].apply(lambda x: x[:4])

In [30]:
df_main

,TobinQ_t1,Stkcd,Accper,ROA_t1,ROE_t1,Pub_year
0,NaN,000001,2020-12-31,0.007383,0.091886,2020
1,NaN,000001,2021-12-31,0.008553,0.104712,2021
2,NaN,000001,2022-12-31,0.008315,0.098353,2022
3,NaN,000001,2023-12-31,NaN,NaN,2023
4,0.929695,000002,2020-12-31,0.019637,0.09546,2020
...,...,...,...,...,...,...
20317,1.397454,900957,2021-12-31,0.038026,0.071539,2021
20318,1.204864,900957,2022-12-31,0.037793,0.066426,2022
20319,NaN,900957,2023-12-31,NaN,NaN,2023
20320,NaN,没有单位,没有单位,NaN,NaN,没有单位


In [37]:
df_main = df_main[df_main['Stkcd'] != '没有单位']
df_main = df_main[df_main['Stkcd'] != '证券代码']

In [38]:
df_main['Stkcd'] = df_main['Stkcd'].astype(int)
df_main['Pub_year'] = df_main['Pub_year'].astype(int)

In [39]:
df2 = df.drop(columns={"Unnamed: 0"}).merge(df_main, left_on=['Corp_code', 'Pub_year'], 
                                      right_on = ['Stkcd', 'Pub_year'])

In [44]:
df2

,Corp_code,Pub_year,Corp_industry_code,Mono_idst,Poll_idst,img_num_pages_ratio,img_size_pages_ratio,dist_diag_ratio,word_simi_avg,Wind_ESG,...,PDF_pages,TobinQ_t-0,TobinQ_t-1,TobinQ_t-2,TobinQ_t-3,TobinQ_t1,Stkcd,Accper,ROA_t1,ROE_t1
0,1,2020,J66,0,0,-1.108526,-0.850690,-1.386487,0.239547,-0.864824,...,83,1.002502,1.001585,0.976896,1.001915,NaN,1,2020-12-31,0.007383,0.091886
1,8,2022,C37,0,0,-0.481239,0.880235,-0.727693,0.308537,-1.046699,...,35,1.165025,1.222200,1.029485,1.197542,1.254821,8,2022-12-31,-0.077376,-0.233197
2,16,2022,C39,0,0,-0.576226,-0.931491,-0.849546,0.299738,-1.188158,...,76,1.003631,1.045699,1.030459,0.972553,1.039644,16,2022-12-31,-0.073577,-0.383352
3,40,2021,D44,1,0,-0.388391,0.025943,0.244892,0.308726,-1.673159,...,42,0.842710,0.809312,0.831874,0.920839,0.837492,40,2021-12-31,-0.012115,-0.027456
4,40,2022,D44,1,0,0.211256,-0.066994,-0.170544,0.386769,0.327471,...,42,0.837492,0.842710,0.809312,0.831874,0.831396,40,2022-12-31,-0.007492,-0.01585
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
901,831039,2022,M74,0,0,0.563307,1.640090,-0.549016,0.518732,-1.279096,...,40,1.159772,1.283682,NaN,NaN,1.966133,831039,2022-12-31,0.077299,0.105347
902,831445,2021,C21,0,0,0.690660,-0.479043,0.362501,0.604800,-0.167634,...,37,3.313189,1.772118,NaN,NaN,1.698036,831445,2021-12-31,0.084971,0.112108
903,835640,2022,C39,0,0,-0.009666,-0.140413,-0.958670,-1.512639,-0.713261,...,19,2.492775,3.764790,1.345553,NaN,2.889939,835640,2022-12-31,0.11296,0.169737
904,838262,2022,C17,0,0,0.278958,-0.616081,0.928865,0.558373,NaN,...,44,1.128583,NaN,NaN,NaN,NaN,838262,2022-12-31,0.066226,0.109402


In [43]:
df2.to_excel('../output/data_stata4_6_0.25_2023.xlsx', index= False)